# GIS Workflow: Historical Cemetery Mapping in Kentucky

## Purpose

This notebook streamlines and automates GIS workflows for historical cemetery mapping in Kentucky. The project is conducted county by county and is sponsored by the Redbird Ranger District, Daniel Boone National Forest, USDA Forest Service, through the Virtual Student Federal Service (VSFS) program.

*Developed by Emily Richardson (VSFS Intern). Published 02/27/2025.*

## Disclaimer

This notebook is a work in progress. While every effort has been made to ensure accuracy and functionality, users should review and adapt the code to fit their specific needs. Additionally, portions of this code have been iteratively refined using AI-assisted development. This process involves generating and testing solutions through structured prompts to improve efficiency and accuracy.

## Overview

1. Getting Started
   - County Selection
   - Directory Setup
   - Download Topographic Maps
   - ArcGIS Pro Project Workspace
      - Add *HistoricalCemeteries_KY* Geodatabase
      - Create and/or Rename a Map
      - Add GeoTIFFs
      - Add County Boundaries
      - Create Cemetery Polygon Feature Class
      - Add Cemetery Point Data
2. Digitization
3. Attribute Data Entry
   - Automated
   - Manual
4. Metadata Creation
5. Sharing

## Data Sources

This project integrates historical and modern datasets from multiple sources to accurately identify and map cemeteries across Kentucky. While the reliability and completeness of these sources vary, they collectively provide a strong foundation for initial cemetery mapping efforts.

### Primary Sources

- **1950s Georeferenced USGS Topographic Maps (1:24,000 scale)**
    - Available for download from:
        - Esri [Historical Topo Map Explorer](https://livingatlas.arcgis.com/topomapexplorer)
        - USGS [topoView](https://ngmdb.usgs.gov/topoview/viewer). (*Downloads require extraction from a zipped folder before use.*)

- **USFS Daniel Boone National Forest (2013 Cemetery Points)**
    - Used to digitize cemeteries not appearing on topographic maps
    - Point feature class: *KY_CemData_Draft2013*

- **USFS Archaeological Cemetery Center Points**
    - Also used to digitize cemeteries absent from topographic maps
    - Point feature class: *Archaeology_HistoricCem_Centerpoints*

- **[Find a Grave](https://www.findagrave.com/cemetery-browse/USA/Kentucky?id=state_19)**
    - Used to identify cemeteries and assign attributes (e.g., "Burial_Number" and "Link")
    - Point feature class: *KYCemPts_FindaGrave_01292025*

- **Zip Code Zones**
    - Available from Esri's [Living Atlas](https://www.arcgis.com/home/item.html?id=5f31109b46d541da86119bd4cf213848) (*updated annually*)
    - Polygon feature class: *zip_zones* (*created using the Living Atlas layer in September 2024*)

- **State and County Boundaries**
    - Sourced from the [US Census Bureau](https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.html)

- **[Google Maps & Google Street View](https://maps.google.com/)**

- **[Kentucky Natural Color Imagery (2024)](https://kygeonet.maps.arcgis.com/home/webmap/viewer.html)**

### Additional Sources

- **[OpenStreetMap](https://www.openstreetmap.org/)**  
- **[Regrid Property App Parcel Data](https://app.regrid.com/us/ky)** (*Used for determining cemetery land ownership and boundaries*)  
- **[Cemeteries in Kentucky Database](https://kyhistory.com/digital/collection/LIB/id/493/)** (*Kentucky Historical Society Library Collection*)  
- **County Historical Societies & Genealogical Associations** (*Local burial records and cemetery maps*)  
- **Local History Books & Manuscripts** (*County histories and cemetery transcription projects*)

#### Online Cemetery Databases

These sources contain cemetery data that may be outdated, incomplete, or inconsistent. Users should cross-reference multiple sources and document concerns in the "Comments" field. When possible, information should be verified using historical maps, property records, or local historical societies.

- **[Billion Graves](https://billiongraves.com/search/cemetery)**
- **[Interment.net](https://www.interment.net/us/ky/index.htm)** 
- **[The Tombstone Transcription Project](http://www.usgwtombstones.org/kentucky/kentucky.html)**
- **[Kentucky Genealogy Cemetery Index](https://kentuckygenealogy.org/cemeteries)**
- **[USGenWeb Archives for Kentucky](http://files.usgwarchives.net/ky/)**
- **[Central Kentucky Cemetery Maps (Lexington Public Library)](https://www.lexpublib.org/cemeteries)**
- **[The Ancestor Hunt – Free Kentucky Cemetery Records](https://theancestorhunt.com/blog/free-kentucky-online-cemetery-and-burial-records/)** 
- **[Churches and Cemeteries in Kentucky](https://churches-and-cemeteries.com/index_pages/KY.html)**
- **[WikiTree Cemetery Category](https://www.wikitree.com/wiki/Category:Kentucky%2C_Cemeteries)**
- **[USGenWeb Archives](http://usgwarchives.net/ky/kyfiles.html)**

## 1️⃣Getting Started

### County Selection

<font color="blue">**Claim a county:**</font> [Kentucky County List](https://docs.google.com/spreadsheets/d/1jMiYkpYpRQLEjVTNRCBKeqTdSbYayahU6MUEkAGZYAU/edit?usp=sharing)

<font color="blue">**Define the County-Quadrangle Dictionary:**</font>

*The countyquads dictionary was manually created using the [Kentucky Geological Survey's Topographic Map Index](https://kgs.uky.edu/kgsweb/download/topo/topoindex.pdf). The dictionary may contain errors due to manual entry. Always verify that the selected quadrangles cover the entire county.*

In [ ]:
# Dictionary mapping Kentucky counties to their respective quadrangles
countyquads = {
    'Adair' : ['Campbellsville', 'Mannsville', 'Clementsville', 'Gresham', 'Cane Valley', 'Knifley', 'Dunnville', 'East Fork', 'Gradyville', 'Columbia', 'Montpelier', 'Russell Springs', 'Breeding', 'Amandaville', 'Creelsboro'],
    'Allen' : ['Polkville', 'Meador', 'Lucas', 'Drake', 'Allen Springs', 'Scottsville', 'Austin', 'Tracy', 'Hickory Flat', 'Adolphus', 'Petroleum', 'Holland', 'Fountain Run'],
    'Anderson' : ['Waddy', 'Frankfort West', 'Mount Eden', 'Glensboro', 'Lawrenceburg', 'Tyrone', 'Chaplin', 'Ashbrook', 'McBrayer', 'Salvisa'],
    'Ballard' : ['Olmsted, IL', 'Bandana', 'Cairo, IL', 'Barlow', 'La Center', 'Heath', 'Wyatt, MO', 'Wickliffe', 'Blandville', 'Lovelaceville'],
    'Barren' : ['Mammoth Cave', 'Horse Cave', 'Park', 'Center', 'Smiths Grove', 'Park City', 'Glasgow North', 'Hiseville', 'Sulphur Well', 'Meador', 'Lucas', 'Glasgow South', 'Temple Hill', 'Summer Shade', 'Austin', 'Tracy', 'Freedom', 'Sulphur Lick', 'Fountain Run'],
    'Bath' : ['Moorefield', 'Sherburne', 'Hillsboro', 'North Middletown', 'Sharpsburg', 'Owingsville', 'Colfax', 'Farmers', 'Preston', 'Olympia', 'Salt Lick', 'Bangor', 'Means', 'Frenchburg'],
    'Bell' : ['Scalf', 'Beverly', 'Helton', 'Artemus', 'Pineville', 'Balkan', 'Wallins Creek', 'Frakes', 'Kayjay', 'Middlesboro North', 'Varilla', 'Ewing', 'Eagan, TN', 'Mingo Mountains, TN', 'Middlesboro South, TN'],
    'Boone' : ['Hooven, OH', 'Addyston, OH', 'Aurora, IN', 'Lawrenceburg, IN', 'Burlington', 'Covington', 'Aberdeen, IN', 'Rising Sun', 'Union', 'Independence', 'Patriot', 'Verona', 'Walton'],
    'Bourbon' : ['Leesburg', 'Shawhan', 'Millersburg', 'Carlisle', 'Centerville', 'Paris West', 'Paris East', 'North Middletown', 'Sharpsburg', 'Lexington East', 'Clintonville', 'Austerlitz', 'Sideview'],
    'Boyd' : ['Ironton, OH', 'Argillite', 'Ashland', 'Catlettsburg, OH', 'Rush', 'Boltsfork', 'Burnaugh', 'Webbville', 'Fallsburg', 'Prichard, WV'],
    'Boyle' : ['Mackville', 'Perryville', 'Danville', 'Bryantsville', 'Gravel Switch', 'Parksville', 'Junction City', 'Stanford'],
    'Bracken' : ['Moscow, OH', 'Felicity, OH', 'Higginsport, OH', 'Berlin', 'Brooksville', 'Germantown', 'Claysville', 'Mount Olivet', 'Sardis'],
    'Breathitt' : ['Campton', 'Landsaw', 'Lee City', 'Seitz', 'Salyersville', 'Tallega', 'Jackson', 'Quicksand', 'Guage', 'Tiptop', 'David', 'Cowcreek', 'Canoe', 'Haddix', 'Noble', 'Vest', 'Handshoe', 'Mistletoe', 'Buckhorn', 'Krypton'],
    'Breckinridge' : ['Derby, IN', 'Alton', 'Rome, IN', 'Lodiburg', 'Irvington', 'Guston', 'Cloverport', 'Mattingly', 'Hardinsburg', 'Garfield', 'Big Spring', 'Fordsville', 'Glen Dean', 'Kingswood', 'Custer', 'Constantine', 'Olaton', 'Falls of Rough', 'McDaniels', 'Madrid'],
    'Bullitt' : ['Kosmosdale, IN', 'Valley Station', 'Brooks', 'Mount Washington', 'Waterford', 'Fort Knox', 'Pitts Point', 'Shepherdsville', 'Samuels', 'Fairfield', 'Colesburg', 'Lebanon Junction', 'Cravens'],
    'Butler' : ['Rosine', 'Spring Lick', 'Cromwell', 'Flener', 'Welchs Creek', 'Ready', 'Rochester', 'South Hill', 'Morgantown', 'Riverside', 'Reedyville', 'Dunmor', 'Sugar Grove', 'Hadley'],
    'Caldwell' : ['Providence', 'Marion', 'Shady Grove', 'Dalton', 'Fredonia', 'Crider', 'Olney', 'Dawson Springs', 'Eddyville', 'Princeton West', 'Princeton East', 'Dawson Springs SW', 'Lamasco', 'Cobb', 'Gracey'],
    'Calloway' : ['Kirksey', 'Dexter', 'Hico', 'Rushing Creek', 'Lynn Grove', 'Hazel', 'New Concord', 'Hamlin', 'Puryear, TN', 'Buchanan, TN', 'Paris Landing, TN'],
    'Campbell' : ['Covington', 'Newport', 'Withamsville, OH', 'Alexandria', 'New Richmond', 'Laurel, OH', 'De Mossville', 'Butler', 'Moscow, OH'],
    'Carlisle' : ['Wickliffe', 'Blandville', 'Lovelaceville', 'Wickliffe SW', 'Arlington', 'Milburn'],
    'Carroll' : ['Vevay North, IN', 'Madison East', 'Carrollton', 'Vevay South', 'Sanders', 'Campbellsburg', 'Worthville'],
    'Carter' : ['Garrison', 'Wesleyville', 'Tygarts Valley', 'Oldtown', 'Argillite', 'Soldier', 'Olive Hill', 'Grahn', 'Grayson', 'Rush', 'Haldeman', 'Ault', 'Brun', 'Willard', 'Webbville'],
    'Casey' : ['Gravel Switch', 'Parksville', 'Junction City', 'Bradfordsville', 'Bradfordsville NE', 'Ellisburg', 'Hustonville', 'Mannsville', 'Clementsville', 'Liberty', 'Yosemite', 'Eubank', 'Dunnville', 'Phil', 'Mintonville', 'Science Hill', 'Eli'],
    'Christian' : ['Dawson Springs', 'Saint Charles', 'Nortonville', 'Graham', 'Dawson Springs SW', 'Dawson Springs SE', 'Crofton', 'Haleys Mill', 'Gracey', 'Pleasant Green Hill', 'Kelly', 'Honey Grove', 'Caledonia', 'Church Hill', 'Hopkinsville', 'Pembroke', 'Roaring Spring', 'Herndon', 'Oak Grove', 'Hammacksville'],
    'Clark' : ['Clintonville', 'Austerlitz', 'Sideview', 'Ford', 'Winchester', 'Hedges', 'Levee', 'Union City', 'Palmer'],
    'Clay' : ['Tyner', 'Maulden', 'Oneida', 'Mistletoe', 'Portersburg', 'Manchester', 'Barcreek', 'Big Creek', 'Blackwater', 'Hima', 'Ogle', 'Creekville', 'Scalf', 'Beverly'],
    'Clinton' : ['Creelsboro', 'Wolf Creek Dam', 'Cumberland City', 'Frogue', 'Albany', 'Savage', 'Powersburg', 'Byrdstown, TN', 'Moodyville, TN', 'Pall Mall, TN'],
    'Crittenden' : ['Dekoven', 'Sturgis', 'Rosiclare, IL', 'Cave In Rock', 'Repton', 'Blackford', 'Providence', 'Lola', 'Salem', 'Marion', 'Shady Grove', 'Dalton', 'Dycusburg', 'Fredonia'],
    'Cumberland' : ['Breeding', 'Amandaville', 'Creelsboro', 'Dubre', 'Waterview', 'Burkesville', 'Wolf Creek Dam', 'Vernon', 'Blacks Ferry', 'Frogue', 'Albany', 'Dale Hollow Dam, TN', 'Dale Hollow Reservoir SE, TN'],
    'Daviess' : ['Yankeetown, IN', 'Rockport, IN', 'Lewisport', 'Reed', 'Owensboro West', 'Owensboro East', 'Maceo', 'Pellville', 'Delaware', 'Curdsville', 'Panther', 'Sutherland', 'Philpot', 'Whitesville', 'Calhoun', 'Glenville', 'Utica', 'Pleasant Ridge'],
    'Edmonson' : ['Ready', 'Bee Spring', 'Nolin Reservoir', 'Cub Run', 'Reedyville', 'Brownsville', 'Rhoda', 'Mammoth Cave', 'Bristow', 'Smiths Grove', 'Park City'],
    'Elliott' : ['Olive Hill', 'Haldeman', 'Ault', 'Bruin', 'Willard', 'Wrigley', 'Sandy Hook', 'Isonville', 'Mazie', 'Lenox', 'Dingus'],
    'Estill' : ['Palmer', 'Clay City', 'Stanton', 'Moberly', 'Panola', 'Irvine', 'Cobhill', 'Zachariah', 'Alcorn', 'Leighton', 'Heidelberg'],
    'Fayette' : ['Georgetown', 'Centerville', 'Paris West', 'Versailles', 'Lexington West', 'Lexington East', 'Clintonville', 'Keene', 'Nicholasville', 'Coletown', 'Ford', 'Valley View', 'Richmond North'],
    'Fleming' : ['Orangeburg', 'Tollesboro', 'Cowan', 'Elizaville', 'Flemingsburg', 'Burtonville', 'Stricklett', 'Moorefield', 'Sherburne', 'Hillsboro', 'Plummers Landing', 'Cranston', 'Colfax', 'Farmers'],
    'Floyd' : ['Paintsville', 'Ivyton', 'Prestonsburg', 'Lancer', 'Thomas', 'David', 'Martin', 'Harold', 'Broad Bottom', 'Handshoe', 'Wayland', 'McDowell', 'Pikeville', 'Kite', 'Wheelwright'],
    'Franklin' : ['Polsgrove', 'Switzer', 'Stamping Ground', 'Waddy', 'Frankfort West', 'Frankfort East', 'Midway', 'Lawrenceburg', 'Tyrone'],
    'Fulton' : ['Bayouville, MO', 'Wolf Island', 'Oakton', 'New Madrid SE, MO', 'Hubbard Lake', 'Bondurant', 'Hickman', 'Cayce', 'Crutchfield', 'Water Valley', 'Point Pleasant, MO', 'Tiptonville, TN'],
    'Gallatin' : ['Vevay North, IN', 'Florence, IN', 'Patriot', 'Verona', 'Vevay South', 'Sanders', 'Glencoe'],
    'Garrard' : ['Wilmore', 'Little Hickman', 'Bryantsville', 'Buckeye', 'Kirksville', 'Stanford', 'Lancaster', 'Paint Lick', 'Berea', 'Brodhead', 'Wildie'],
    'Grant' : ['Patriot', 'Verona', 'Walton', 'Glencoe', 'Elliston', 'Williamstown', 'Goforth', 'Owenton', 'Lawrenceville', 'Mason', 'Berry', 'New Columbus', 'Sadieville'],
    'Graves' : ['Lovelaceville', 'Melber', 'Symsonia', 'Elva', 'Fancy Farm', 'Hickory', 'Westplains', 'Oak Level', 'Dublin', 'Mayfield', 'Farmington', 'Kirksey', 'Water Valley', 'Cuba', 'Lynnville', 'Lynn Grove'],
    'Grayson' : ['Olaton', 'Falls of Rough', 'McDaniels', 'Madrid', 'Big Clifty', 'Summit', 'Rosine', 'Spring Lick', 'Caneyville', 'Leitchfield', 'Clarkson', 'Millerstown', 'Welchs Creek', 'Ready', 'Bee Spring', 'Nolin Reservoir', 'Cub Run'],
    'Green' : ['Magnolia', 'Hibernia', 'Saloma', 'Hudgins', 'Summersville', 'Greensburg', 'Center', 'Exie', 'Gresham', 'Cane Valley', 'Sulphur Well', 'East Fork', 'Gradyville'],
    'Greenup' : ['New Boston, OH', 'Friendship', 'Portsmouth', 'Wheelersburg, OH', 'Garrison', 'Brushart', 'Load', 'Greenup', 'Ironton, OH', 'Wesleyville', 'Tygarts Valley', 'Oldtown', 'Argillite', 'Ashland', 'Rush'],
    'Hancock' : ['Lewisport', 'Tell City', 'Cannelton, IN', 'Maceo', 'Pellville', 'Cloverport', 'Whitesville', 'Fordsville', 'Glen Dean'],
    'Hardin' : ['Kosmosdale, IN', 'Fort Knox', 'Pitts Point', 'Big Spring', 'Flaherty', 'Vine Grove', 'Colesburg', 'Lebanon Junction', 'Custer', 'Constantine', 'Howe Valley', 'Cecilia', 'Elizabethtown', 'Nelsonville', 'Madrid', 'Big Clifty', 'Summit', 'Sonora', 'Tonieville', 'Millerstown', 'Upton'],
    'Harlan' : ['Leatherwood', 'Tilford', 'Roxana', 'Helton', 'Bledsoe', 'Nolansburg', 'Louellen', 'Benham', 'Appalachia, VA', 'Balkan', 'Wallins Creek', 'Harlan', 'Evarts', 'Pennington Gap, VA', 'Keokee, VA', 'Varilla', 'Ewing', 'Rose Hill, VA', 'Hubbard Springs, VA'],
    'Harrison' : ['Mason', 'Berry', 'Kelat', 'Claysville', 'Sadieville', 'Breckinridge', 'Cynthiana', 'Shady Nook', 'Piqua', 'Leesburg', 'Shawhan', 'Millersburg'],
    'Hart' : ['Millerstown', 'Upton', 'Hammonville', 'Magnolia', 'Nolin Reservoir', 'Cub Run', 'Munfordville', 'Canmer', 'Hudgins', 'Mammoth Cave', 'Horse Cave', 'Park', 'Center'],
    'Henderson' : ['Mount Vernon, IN', 'Caborn, IN', 'West Franklin, IN', 'Evansville, IN', 'Newburgh, IN', 'Yankeetown, IN', 'Uniontown', 'Smith Mills', 'Wilson', 'Henderson', 'Spottsville', 'Reed', 'Waverly', 'Poole', 'Robards', 'Delaware', 'Curdsville'],
    'Henry' : ['Bedford', 'Campbellsburg', 'Worthville', 'New Liberty', 'Smithfield', 'New Castle', 'Franklinton', 'Gratz', 'Ballardsville', 'Eminence', 'North Pleasureville', 'Polsgrove', 'Switzer'],
    'Hickman' : ['Wickliffe SW', 'Arlington', 'Milburn', 'Fancy Farm', 'Wolf Island', 'Oakton', 'Clinton', 'Dublin', 'Cayce', 'Crutchfield', 'Water Valley'],
    'Hopkins' : ['Beech Grove', 'Calhoun', 'Providence', 'Nebo', 'Slaughters', 'Hanson', 'Sacramento', 'Dalton', 'Coiltown', 'Madisonville West', 'Madisonville East', 'Millport', 'Olney', 'Dawson Springs', 'Saint Charles', 'Nortonville', 'Graham' ,'Dawson Springs SE', 'Crofton'],
    'Jackson' : ['Bighill', 'Alcorn', 'Leighton', 'Johnetta', 'Sandgap', 'McKee', 'Sturgeon', 'Livingston', 'Parrot', 'Tyner', 'Maulden'],
    'Jefferson' : ['Charlestown, IN', 'New Albany, IN', 'Jeffersonville, IN', 'Anchorage', 'Crestwood', 'Lanesville, IN', 'Louisville West', 'Louisville East', 'Jeffersontown', 'Fisherville', 'Kosmosdale, IN', 'Valley Station', 'Brooks', 'Mount Washington', 'Waterford', 'Fort Knox'],
    'Jessamine' : ['Versailles', 'Keene', 'Nicholasville', 'Coletown', 'Wilmore', 'Little Hickman', 'Valley View', 'Buckeye'],
    'Johnson' : ['Mazie', 'Redbush', 'Sitka', 'Richardson', 'Milo', 'Salyersville North', 'Oil Springs', 'Paintsville', 'Offutt', 'Inez', 'Ivyton', 'Prestonsburg', 'Lancer'],
    'Kenton' : ['Covington', 'Newport', 'Independence', 'Alexandria', 'Walton', 'De Mossville'],
    'Knott' : ['Tiptop', 'David', 'Noble', 'Vest', 'Handshoe', 'Wayland', 'Hazard North', 'Carrie', 'Hindman', 'Kite', 'Wheelwright', 'Hazard South', 'Vicco', 'Blackey', 'Mayking'],
    'Knox' : ['Blackwater', 'Hima', 'Ogle', 'Corbin', 'Heidrick', 'Fount', 'Scalf', 'Beverly', 'Rockholds', 'Barbourville', 'Artemus', 'Pineville', 'Frakes', 'Kayjay'],
    'LaRue' : ['Elizabethtown', 'Nelsonville', 'New Haven', 'Sonora', 'Tonieville', 'Hodgenville', 'Howardstown', 'Raywick', 'Upton', 'Hammonville', 'Magnolia', 'Hibernia', 'Saloma'],
    'Laurel' : ['Livingston', 'Parrot', 'Tyner', 'Billows', 'Bernstadt', 'London', 'Portersburg', 'Ano', 'London SW', 'Lily', 'Blackwater', 'Hima', 'Sawyer', 'Vox', 'Corbin', 'Heidrick'],
    'Lawrence' : ['Boltsfork', 'Burnaugh', 'Willard', 'Webbville', 'Fallsburg', 'Prichard, WV', 'Isonville', 'Mazie', 'Blaine', 'Adams', 'Louisa', 'Dingus', 'Redbush', 'Sitka', 'Richardson', 'Milo', 'Webb, WV'],
    'Lee' : ['Cobhill', 'Zachariah', 'Campton', 'Leighton', 'Heidelberg', 'Beattyville', 'Tallega', 'Sturgeon', 'Booneville'],
    'Leslie' : ['Buckhorn', 'Krypton', 'Big Creek', 'Hyden West', 'Hyden East', 'Hazard South', 'Creekville', 'Hoskinston', 'Cutshin', 'Leatherwood', 'Beverly', 'Helton', 'Beldsoe', 'Nolansburg'],
    'Letcher' : ['Kite', 'Wheelwright', 'Vicco', 'Blackey', 'Mayking', 'Jenkins West', 'Jenkins East', 'Tilford', 'Roxana', 'Whitesburg', 'Flat Gap, VA', 'Nolansburg', 'Louellen', 'Benham', 'Appalachia, VA'],
    'Lewis' : ['Maysville East, OH', 'Machester Islands', 'Concord, OH', 'Buena Vista, OH', 'Pond Run, OH', 'Friendship', 'Orangeburg', 'Tollesboro', 'Charters', 'Vanceburg', 'Garrison', 'Brushart', 'Burtonville', 'Stricklett', 'Head of Grassy', 'Wesleyville', 'Cranston', 'Soldier', 'Olive Hill'],
    'Lincoln' : ['Bryantsville', 'Junction City', 'Stanford', 'Lancaster', 'Paint Lick', 'Hustonville', 'Halls Gap', 'Crab Orchard', 'Brodhead', 'Yosemite', 'Eubank', 'Woodstock', 'Maretburg', 'Science Hill'],
    'Livingston' : ['Shetlerville, IL', 'Rosiclare, IL', 'Brownfield, IL', 'Golconda', 'Lola', 'Salem', 'Smithland', 'Burna', 'Dycusburg', 'Paducah East', 'Little Cypress', 'Calvert City', 'Grand Rivers', 'Briensburg', 'Birmingham Point'],
    'Logan' : ['Rosewood', 'Dunmor', 'Quality', 'Sugar Grove', 'Sharon Grove', 'Lewisburg', 'Homer', 'South Union', 'Rockfield', 'Olmstead', 'Russellville', 'Dennis', 'Auburn', 'Allensville', 'Dot', 'Adairville', 'Prices Mill'],
    'Lyon' : ['Dycusburg', 'Fredonia', 'Grand Rivers', 'Eddyville', 'Princeton West', 'Birmingham Point', 'Mont', 'Lamasco', 'Fairdealing'],
    'Madison' : ['Coletown', 'Ford', 'Winchester', 'Little Hickman', 'Valley View', 'Richmond North', 'Union City', 'Palmer', 'Buckeye', 'Kirksville', 'Richmond South', 'Moberly', 'Panola', 'Paint Lick', 'Berea', 'Bighill', 'Alcorn'],
    'Magoffin' : ['Lenox', 'Dingus', 'Cannel City', 'White Oak', 'Salyersville North', 'Oil Springs', 'Lee City', 'Seitz', 'Salyersville South', 'Ivyton', 'Guage', 'Tiptop', 'David', 'Handshoe'],
    'Marion' : ['Loretto', 'Saint Catharine', 'Springfield', 'Mackville', 'Howardstown', 'Raywick', 'Lebanon West', 'Lebanon East', 'Gravel Switch', 'Saloma', 'Spurlington', 'Bradfordsville', 'Bradfordsville NE'],
    'Marshall' : ['Little Cypress', 'Calvert City', 'Elva', 'Briensburg', 'Birmingham Point', 'Oak Level', 'Hardin', 'Fairdealing', 'Fenton', 'Kirksey', 'Dexter', 'Hico', 'Rushing Creek'],
    'Martin' : ['Milo', 'Webb, WV', 'Offutt', 'Inez', 'Kermit', 'Naugatuck, WV', 'Lancer', 'Thomas', 'Varney', 'Williamson'],
    'Mason' : ['Higginsport, OH', 'Russellville, OH', 'Germantown', 'Maysville West', 'Maysville East, OH', 'Sardis', 'Mays Lick', 'Orangeburg', 'Tollesboro', 'Cowan', 'Elizaville', 'Flemingsburg'],
    'McCracken' : ['Bandana', 'Joppa, IL', 'Metropolis, IL', 'La Center', 'Heath', 'Paducah West', 'Paducah East', 'Little Cypress', 'Lovelaceville', 'Melber', 'Symsonia', 'Elva'],
    'McCreary' : ['Burnside', 'Hail', 'Sawyer', 'Coopersville', 'Nevelsville', 'Wiborg', 'Cumberland Falls', 'Parmleysville', 'Bell Farm', 'Barthell', 'Whitley City', 'Hollyhill', 'Sharp Place, TN', 'Barthell SW, TN', 'Oneida North, TN', 'Winfield, TN', 'Ketchen, TN', 'Jellico West, TN'],
    'McLean' : ['Delaware', 'Curdsville', 'Beech Grove', 'Calhoun', 'Glenville', 'Utica', 'Sacramento', 'Livermore', 'Equality'],
    'Meade' : ['Beechwood, IN', 'Leavenworth, IN', 'Alton', 'New Amsterdam', 'Mauckport, IN', 'Laconia, IN', 'Lodiburg', 'Irvington', 'Guston', 'Rock Haven', 'Fort Knox', 'Big Spring', 'Flaherty', 'Vine Grove'],
    'Menifee' : ['Olympia', 'Salt Lick', 'Bangor', 'Means', 'Frenchburg', 'Scranton', 'Ezel', 'Slade', 'Pomeroyton', 'Hazel Green'],
    'Mercer' : ['Ashbrook', 'McBrayer', 'Salvisa', 'Cardwell', 'Cornishville', 'Harrodsburg', 'Wilmore', 'Mackville', 'Perryville', 'Danville', 'Bryantsville'],
    'Metcalfe' : ['Park', 'Center', 'Hiseville', 'Sulphur Well', 'East Fork', 'Gradyville', 'Summer Shade', 'Edmonton', 'Breeding', 'Sulphur Lick', 'Dubre', 'Waterview'],
    'Monroe' : ['Tracy', 'Freedom', 'Sulphur Lick', 'Dubre', 'Fountain Run', 'Gamaliel', 'Tompkinsville', 'Vernon', 'Blacks Ferry', 'Galen, TN', 'Red Boiling Springs, TN', 'Union Hill, TN', 'Celina, TN', 'Dale Hollow Dam, TN'],
    'Montgomery' : ['North Middletown', 'Sharpsburg', 'Owingsville', 'Sideview', 'Mount Sterling', 'Preston', 'Hedges', 'Levee', 'Means'],
    'Morgan' : ['Bangor', 'Wrigley', 'Sandy Hook', 'Isonville', 'Ezel', 'West Liberty', 'Lenox', 'Dingus', 'Redbush', 'Pomeroyton', 'Hazel Green', 'Cannel City', 'White Oak', 'Salyersville North', 'Oil Springs', 'Lee City', 'Seitz'],
    'Muhlenberg' : ['Sacramento', 'Livermore', 'Equality', 'Madisonville East', 'Millport', 'Central City West', 'Central City East', 'Paradise', 'Graham', 'Greenville', 'Drakesboro', 'Rochester', 'Haleys Mill', 'Kirkmansville', 'Rosewood', 'Dunmor'],
    'Nelson' : ['Samuels', 'Fairfield', 'Bloomfield', 'Chaplin', 'Lebanon Junction', 'Cravens', 'Bardstown', 'Maud', 'Brush Grove', 'Nelsonville', 'New Haven', 'Loretto', 'Saint Catharine', 'Hodgenville', 'Howardstown', 'Raywick'],
    'Nicholas' : ['Shady Nook', 'Piqua', 'Cowan', 'Millersburg', 'Carlisle', 'Moorefield', 'Sherburne', 'North Middletown', 'Sharpsburg'],
    'Ohio' : ['Philpot', 'Whitesville', 'Fordsville', 'Utica', 'Pleasant Ridge', 'Dundee', 'Olaton', 'Falls of Rough', 'Livermore', 'Equality', 'Hartford', 'Horton', 'Rosine', 'Spring Lick', 'Central City West', 'Central City East', 'Paradise', 'Cromwell', 'Flener', 'Rochester', 'South Hill'],
    'Oldham' : ['Bethlehem, IN', 'Bedford', 'Charlestown, IN', 'Owen', 'La Grange', 'Smithfield', 'Jeffersonville, IN', 'Anchorage', 'Crestwood', 'Ballardsville'],
    'Owen' : ['Vevay South', 'Sanders', 'Glencoe', 'Worthville', 'New Liberty', 'Owenton', 'Lawrenceville', 'Gratz', 'Monterey', 'New Columbus', 'Sadieville', 'Polsgrove', 'Switzer', 'Stamping Ground'],
    'Owsley' : ['Leighton', 'Heidelberg', 'Beattyville', 'Tallega', 'McKee', 'Sturgeon', 'Booneville', 'Cowcreek', 'Maulden', 'Oneida', 'Mistletoe'],
    'Pendleton' : ['Walton', 'De Mossville', 'Butler', 'Moscow, OH', 'Williamstown', 'Goforth', 'Falmouth', 'Berlin', 'Berry', 'Kelat', 'Claysville'],
    'Perry' : ['Canoe', 'Haddix', 'Noble', 'Vest', 'Mistletoe', 'Buckhorn', 'Krypton', 'Hazard North', 'Carrie', 'Big Creek', 'Hyden West', 'Hyden East', 'Hazard South', 'Vicco', 'Leatherwood', 'Tilford', 'Louellen'],
    'Pike' : ['Thomas', 'Varney', 'Williamson', 'Delbarton, WV', 'Broad Bottom', 'Meta', 'Belfry', 'Matewan', 'Majestic, WV', 'Wharncliffe, WV', 'McDowell', 'Pikeville', 'Millard', 'Lick Creek', 'Jamboree', 'Hurley, VA', 'Wheelwright', 'Dorton', 'Hellier', 'Elkhorn City', 'Harman, VA', 'Jenkins West', 'Jenkins East', 'Clintwood'],
    'Powell' : ['Hedges', 'Levee', 'Means', 'Frenchburg', 'Palmer', 'Clay City', 'Stanton', 'Slade', 'Cobhill', 'Zachariah'],
    'Pulaski' : ['Eubank', 'Woodstock', 'Maretburg', 'Phil', 'Mintonville', 'Science Hill', 'Bobtown', 'Shopville', 'Billows', 'Eli', 'Faubush', 'Delmer', 'Somerset', 'Dykes', 'Ano', 'Mill Springs', 'Frazer', 'Burnside', 'Hail', 'Sawyer', 'Nevelsville'],
    'Robertson' : ['Claysville', 'Mount Olivet', 'Sardis', 'Shady Nook', 'Piqua', 'Cowan'],
    'Rockcastle' : ['Berea', 'Bighill', 'Brodhead', 'Wildie', 'Johnetta', 'Woodstock', 'Maretburg', 'Mount Vernon', 'Livingston', 'Shopville', 'Billows', 'Bernstadt'],
    'Rowan' : ['Stricklett', 'Plummers Landing', 'Cranston', 'Soldier', 'Colfax', 'Farmers', 'Morehead', 'Haldeman', 'Ault', 'Salt Lick', 'Bangor', 'Wrigley'],
    'Russell' : ['Dunnville', 'Phil', 'Montpelier', 'Russell Springs', 'Eli', 'Faubush', 'Amandaville', 'Creelsboro', 'Jamestown', 'Jabez', 'Mill Springs', 'Wolf Breek Dam', 'Cumberland City'],
    'Scott' : ['New Columbus', 'Sadieville', 'Breckinridge', 'Stamping Ground', 'Delaplain', 'Leesburg', 'Midway', 'Georgetown', 'Centerville', 'Versailles', 'Lexington West'],
    'Shelby' : ['Crestwood', 'Ballardsville', 'Eminence', 'North Pleasureville', 'Polsgrove', 'Fisherville', 'Simpsonville', 'Shelbyville', 'Waddy', 'Taylorsville', 'Mount Eden', 'Glensboro'],
    'Simpson' : ['South Union', 'Rockfield', 'Auburn', 'Woodburn', 'Drake', 'Adairville', 'Prices Mill', 'Franklin', 'Hickory Flat'],
    'Spencer' : ['Fisherville', 'Simpsonville', 'Mount Washington', 'Waterford', 'Taylorsville', 'Mount Eden', 'Glensboro', 'Samuels', 'Fairfield', 'Bloomfield', 'Chaplin'],
    'Taylor' : ['Hibernia', 'Saloma', 'Spurlington', 'Bradfordsville', 'Bradfordsville NE', 'Greensburg', 'Campbellsville', 'Mannsville', 'Gresham', 'Cane Valley'],
    'Todd' : ['Haleys Mill', 'Kirkmansville', 'Rosewood', 'Honey Grove', 'Allegre', 'Sharon Grove', 'Pembroke', 'Elkton', 'Olmstead', 'Hammacksville', 'Guthrie', 'Allensville'],
    'Trigg' : ['Dawson Springs', 'Birmingham Point', 'Mont', 'Lamasco', 'Cobb', 'Gracey', 'Fairdealing', 'Fenton', 'Canton', 'Cadiz', 'Caledonia', 'Rushing Creek', 'Model', 'Johnson Hollow', 'Roaring Spring'],
    'Trimble' : ['Madison West, IN', 'Madison East', 'Carrollton', 'Bethlehem, IN', 'Bedford', 'Campbellsburg', 'Smithfield'],
    'Union' : ['Mount Vernon, IN', 'Wabash Island, IL', 'Uniontown', 'Smith Mills', 'Shawneetown, IL', 'Grove Center', 'Morganfield', 'Waverly', 'Poole', 'Saline Mines, IL', 'Dekoven', 'Sturgis', 'Bordley', 'Blackford'],
    'Warren' : ['Morgantown', 'Riverside', 'Reedyville', 'Brownsville', 'Sugar Grove', 'Hadley', 'Bowling Green North', 'Bristow', 'Smiths Grove', 'Park City', 'South Union', 'Rockfield', 'Bowling Green South', 'Polkville', 'Meador', 'Woodburn', 'Drake', 'Allen Springs'],
    'Washington' : ['Chaplin', 'Ashbrook', 'Bardstown', 'Maud', 'Brush Grove', 'Cardwell', 'Loretto', 'Saint Catharine', 'Springfield', 'Mackville'],
    'Wayne' : ['Jamestown', 'Jabez', 'Mill Springs', 'Frazer', 'Burnside', 'Cumberland City', 'Parnell', 'Monticello', 'Coopersville', 'Nevelsville', 'Savage', 'Powersburg', 'Parmleysville', 'Bell Farm', 'Pall Mall, TN', 'Sharp Place, TN'],
    'Webster' : ['Poole', 'Robards', 'Delaware', 'Sturgis', 'Bordley', 'Dixon', 'Sebree', 'Beech Grove', 'Blackford', 'Providence', 'Nebo', 'Slaughters', 'Hanson', 'Dalton'],
    'Whitley' : ['Sawyer', 'Vox', 'Corbin', 'Cumberland Falls', 'Wofford', 'Rockholds', 'Barbourville', 'Hollyhill', 'Williamsburg', 'Saxton', 'Frakes', 'Ketchen, TN', 'Jellico West, TN', 'Jellico East, TN', 'Eagan, TN'],
    'Wolfe' : ['Slade', 'Pomeroyton', 'Hazel Green', 'Cannel City', 'Zachariah', 'Campton', 'Landsaw', 'Lee City', 'Seitz', 'Jackson'],
    'Woodford' : ['Frankfort East', 'Midway', 'Tyrone', 'Versailles', 'Salvisa', 'Keene', 'Harrodsburg', 'Wilmore']
    }

<font color="blue">**Create a "county" variable:**</font>

In [ ]:
while True:
    county = input("What county are you working on? (Press enter to exit): ")
    
    if county == "":
        print("Terminating script.")
        break
    elif county in countyquads:
        print(f"You have selected \033[1m{county}\033[0m County.")
        break
    else:
        print("The county you entered is not in the dictionary. Please try again.")

### Directory Setup

<font color="blue">**Create a new directory structure, either inside an existing "topos" directory or within a newly created "topos" directory, and format the subdirectory names based on the current date and county name:**</font>

*How It Works:*
- *Prompts the user to decide if an existing "topos" directory should be used or a new one should be created.*
- *If an existing "topos" directory is chosen, it generates a subdirectory with a formatted name based on the county name, current date, and the next available number.*
- *If a new "topos" directory is created, the script sets up the "topos" directory first and then creates the same county subdirectory with the appropriate naming convention.*

In [ ]:
import os
import tkinter as tk
from tkinter import filedialog
from datetime import datetime

def format_dir_name(name):
    """Format directory names by replacing spaces with hyphens and commas with underscores."""
    name = name.lower().replace(', ', '_').replace(' ', '-')
    name = name.replace('-_', '_') # Ensure no mixed characters like "-_"
    return name

def get_next_directory_number(base_directory):
    """Determine the next available directory number in the given base directory."""
    existing_dirs = os.listdir(base_directory)
    existing_numbers = []

    # Loop through existing directories and find those starting with a number
    for directory in existing_dirs:
        # Check if the directory starts with a number and a hyphen (e.g., "01-whitley")
        if directory[0].isdigit() and '-' in directory:
            # Extract the number from the beginning of the directory name
            number_part = directory.split('-')[0]
            if number_part.isdigit():
                existing_numbers.append(int(number_part))
    
    # Find the next available number (starting from 1)
    next_number = 1
    if existing_numbers:
        next_number = max(existing_numbers) + 1

    # Format the number as two digits (01, 02, etc.)
    return f"{next_number:02d}"

def get_current_date():
    """Return the current date formatted as MMDDYYYY."""
    return datetime.now().strftime("%m%d%Y")

# Initialize GUI file dialog
root = tk.Tk()
root.withdraw()

directory_order = input('Do you have an existing "topos" directory? (y/n) ').lower()

current_date = get_current_date()

if directory_order == "y":
    print(f'Select the existing "topos" directory.')
    topo_directory = filedialog.askdirectory()
    
    # Format the county name
    formatted_county = format_dir_name(county)
    
    # Get the next available number for the county directory
    next_number = get_next_directory_number(topo_directory)
    new_county_directory_name = f"{next_number}-{formatted_county}_{current_date}"
    county_directory = os.path.join(topo_directory, new_county_directory_name)
    
    # Create the county subdirectory
    os.makedirs(county_directory, exist_ok=True)
    print(f'Success! Created: "{os.path.normpath(county_directory)}')

elif directory_order == "n":
    print(f'Select the parent directory to create a new "topos" directory.')
    directory_path = filedialog.askdirectory()

    # Create "topos" directory
    topo_directory = os.path.join(directory_path, "topos")
    os.makedirs(topo_directory, exist_ok=True)
    print(f'Subdirectory "topos" created at: {os.path.normpath(topo_directory)}.')
    
    # Format the county name
    formatted_county = format_dir_name(county)
    
    # Get the next available number for the county directory
    next_number = get_next_directory_number(topo_directory)
    new_county_directory_name = f"{next_number}-{formatted_county}_{current_date}"
    county_directory = os.path.join(topo_directory, new_county_directory_name)
    
    # Create the county subdirectory
    os.makedirs(county_directory, exist_ok=True)
    print(f'Subdirectory "{os.path.normpath(county_directory)}" created.')

else:
    print("Invalid input. Please re-run and enter 'y' or 'n'.")

<font color="blue">**Connect the "topos" folder to the project:**</font>

In [ ]:
import arcpy
import os

# Get the path of the currently open ArcGIS Pro project (if there is one open)
aprx = arcpy.mp.ArcGISProject("CURRENT")  # 'CURRENT' refers to the active project

# Check if the project is open
if not aprx:
    print("No project is currently open.")
else:
    # Get the current folder connections
    folder_connections = aprx.folderConnections

    # Define the new folder path using os.path.normpath
    new_folder = os.path.normpath(topo_directory)  # Normalize the path
    alias = "Topos Folder"  # Alias for the new folder connection
    is_home_folder = False  # Set to True if this should be the default home folder

    # Create a dictionary for the new folder connection
    new_folder_dict = {
        "connectionString": new_folder,
        "alias": alias,
        "isHomeFolder": is_home_folder
    }

    # Add the new folder connection to the list
    folder_connections.append(new_folder_dict)

    # Optional: Validate the folder path (ensure the folder exists if validate=True)
    validate = True

    # Update the folder connections in the project
    aprx.updateFolderConnections(folder_connections, validate)

    # Save the project after the update
    aprx.save()

    print("Folder connection updated successfully!")

### Download Topographic Maps

This project uses U.S. Geological Survey topographic maps from the 1950s (1:24,000 scale). If 1950s maps are not available, late 1940s or early 1960s maps may be substituted. Users have the option to manually download GeoTIFFs from either Esri's [Historical Topo Map Explorer](https://livingatlas.arcgis.com/topomapexplorer) or USGS's [topoView](https://ngmdb.usgs.gov/topoview/viewer).

Esri's source allows for direct download of GeoTIFFs, whereas downloads from the USGS tool must be extracted from a zipped folder before use. For this reason, the script below utilizes Esri Explorer.

**It may also be useful to download current topographic maps from USGS's topoView.** Newer maps may include cemetery names that were absent on older maps, especially those that have expanded considerably since the 1950s. Additionally, some cemeteries may no longer exist due to changes in the landscape, such as the creation of reservoirs. Although these changes can be observed on Google Maps or through the [KyFromAbove Program imagery](https://kygeonet.maps.arcgis.com/home/webmap/viewer.html?webmap=ba05e691cf3a4acd9583b12ccf09856e), in some cases, comparing topographic layers in ArcGIS Pro by toggling layers on and off is more effective.

<font color="blue">**Generate URLs to access topographic maps for the quadrangles that cover the county of interest:**</font>

*How It Works:*
- *Most quadrangles are named after a prominent place within the region.*
- *The script checks if the county is present in a predefined dictionary and retrieves the corresponding quadrangle names.*
- *For each quadrangle, the script queries the Nominatim API for geographic coordinates and generates an Esri map URL for the 1:24,000 scale topographic maps.*
- *It outputs the map URLs with formatting and provides instructions for downloading the maps.*

In [ ]:
import urllib.parse
import urllib.request
import json

# ANSI escape codes for red text and bold formatting
RED = '\033[91m'
BOLD = '\033[1m'
UNDERLINE = '\033[4m'
RESET = '\033[0m'

# Function to get coordinates from place name
def get_coordinates(place_name):
    # Check if the place name already has a state abbreviation
    if ', ' not in place_name or len(place_name.split(', ')[1]) != 2:
        place_name += ", KY"  # Add KY for Kentucky places without state code
    
    # Construct the URL for Nominatim API
    base_url = "https://nominatim.openstreetmap.org/search?"
    
    # Define parameters
    params = {
        'q': place_name,
        'format': 'json',
        'addressdetails': 1,
        'limit': 1
    }
    
    # Encode parameters and create the full URL
    url = base_url + urllib.parse.urlencode(params)
    
    # Make the API request and get the response
    with urllib.request.urlopen(url) as response:
        data = json.load(response)
    
    if data:
        lat = data[0]['lat']
        lon = data[0]['lon']
        return lat, lon
    else:
        return None, None

# Function to generate the Esri URL (now only showing Esri URL for 24k maps)
def generate_map_link(lat, lon):
    # Esri URL for 24k (1:24,000) scale with LoD set to 15.76
    esri_url = f"https://livingatlas.arcgis.com/topomapexplorer/#maps=&loc={lon},{lat}&LoD=15.76"
    return esri_url

# Assuming 'county' has already been defined somewhere earlier in the notebook

# Check if the 'county' is in the countyquads dictionary
if county in countyquads.keys():
    quads = countyquads[county]
    
    # Print the introductory message in red, including the number of maps in bold
    print(f"{RED}Please open the URL for each quadrangle, filter for the 24k (1:24,000) scale, and download the appropriate topographic map for the year you want (~1950s).")
    print(f"{RESET}{BOLD}{county} County requires {UNDERLINE}{len(quads)}{RESET}{BOLD} topographic maps:{RESET}")
    
    # Iterate over each quadrangle name
    for quad in quads:
        lat, lon = get_coordinates(quad)
        
        if lat and lon:
            # Generate the Esri map link
            esri_url = generate_map_link(lat, lon)
            
            # Print the result with the quadrangle name in bold, pipe separator, and the Esri URL
            print(f"{BOLD}{quad}{RESET} | {esri_url}")
        else:
            print(f"Coordinates for {quad} not found. Please open the topo map explorer and manually search for {quad}.")
else:
    print("County not found in the predefined list.\nPlease make sure the county name is correct.")

### ArcGIS Pro Project Workspace

#### Add *HistoricalCemeteries_KY* Geodatabase

<font color="blue">**Select and add the *HistoricalCemeteries_KY* geodatabase to the project:**</font>

*How It Works:*
- *The user selects a geodatabase folder via a file dialog using Tkinter.*
- *The script checks if the selected folder is a valid geodatabase and if the project is not read-only.*
- *The selected geodatabase is added to the project, the databases list is updated, and the project is saved.*

In [ ]:
import arcpy
import os
import tkinter as tk
from tkinter import filedialog

# Initialize Tkinter window (for file dialog)
root = tk.Tk()
root.withdraw()  # Hides the root window

# Prompt user to select a geodatabase
gdb_path = filedialog.askdirectory(title="Select HistoricalCemeteries_KY.gdb")

# Ensure the path is a valid geodatabase
if gdb_path and os.path.isdir(gdb_path) and gdb_path.lower().endswith('.gdb'):
    # Get the active ArcGIS Pro project
    aprx = arcpy.mp.ArcGISProject("CURRENT")  # Refers to the active project
    
    # Check if the project is read-only
    if aprx.isReadOnly:
        print('WARNING: Project is read-only. Exiting.')
        exit()
    
    # Get the current list of databases in the project
    db = aprx.databases
    
    # Append the selected geodatabase to the project (not as default)
    db.append({'databasePath': gdb_path, 'isDefaultDatabase': False})
    
    # Update the project's databases list
    aprx.updateDatabases(db, True)
    
    # Save the project after making the changes
    aprx.save()
    
    print(f"Geodatabase '{gdb_path}' added to the project.")
else:
    print("Invalid selection or no geodatabase selected. Exiting.")

<font color="blue">**Make *HistoricalCemeteries_KY.gdb* the default database:**</font>

In [ ]:
import arcpy

# Get the active ArcGIS project
aprx = arcpy.mp.ArcGISProject("CURRENT")

# Set the default geodatabase
aprx.defaultGeodatabase = gdb_path

# Save the project to apply the change
aprx.save()

# Print confirmation
print(f"Default geodatabase set to: {gdb_path}")

#### Create and/or Rename a Map

<font color="blue">**Rename an existing map or create a new map:**</font>

*How It Works:*
- *The script lists all existing maps in the current ArcGIS Pro project, allowing the user to choose either to rename an existing map or create a new one.*
- *If creating a new map, the script checks for any naming conflicts and prompts the user to provide a new name if needed.*
- *The selected map is renamed based on a predefined county variable.*

In [ ]:
import arcpy

# Function to list all maps
def list_all_maps():
    try:
        aprx = arcpy.mp.ArcGISProject("CURRENT")
        maps = aprx.listMaps()
        return maps
    except Exception as e:
        arcpy.AddMessage(f"Error: {str(e)}")
        return []

# Function to rename an existing map
def rename_map(map_obj, new_name):
    try:
        map_obj.name = new_name
        arcpy.AddMessage(f"Map renamed to {new_name}")
    except Exception as e:
        arcpy.AddMessage(f"Error renaming map: {str(e)}")

# Function to create a new map
def create_new_map(aprx):
    # Check if a map with the same name exists
    maps = aprx.listMaps()
    new_map_name = f"{county}_County"
    
    # Check for duplicate names and prompt user if necessary
    existing_map_names = [map_obj.name for map_obj in maps]
    if new_map_name in existing_map_names:
        user_choice = input(f"A map with the name '{new_map_name}' already exists. Would you like to rename the new map? (y/n): ")
        if user_choice.lower() == "y":
            new_map_name = input("Enter a new name for the new map: ")
        else:
            arcpy.AddMessage("New map creation canceled.")
            return None
    
    try:
        new_map = aprx.createMap("New Map")
        # Rename the new map to the determined name
        rename_map(new_map, new_map_name)
        arcpy.AddMessage(f"New map created and renamed: {new_map.name}")
        return new_map
    except Exception as e:
        arcpy.AddMessage(f"Error creating new map: {str(e)}")
        return None

# Function to handle the renaming or creation process
def rename_or_create_map():
    aprx = arcpy.mp.ArcGISProject("CURRENT")
    maps = list_all_maps()

    # List all existing maps
    if len(maps) > 0:
        print("The following maps exist:")
        for idx, map_obj in enumerate(maps):
            print(f"{idx + 1}. {map_obj.name}")
    
    # Always ask the user if they want to create a new map
    user_choice = input(f"Which map would you like to rename (1-{len(maps)})? Or, enter '0' to create a new map: ")

    if user_choice == '0':
        # Create a new map and rename it to county_County
        return create_new_map(aprx), True
    else:
        try:
            selected_map = maps[int(user_choice) - 1]
            rename_map(selected_map, f"{county}_County")
            return selected_map, True
        except (ValueError, IndexError):
            arcpy.AddMessage("Invalid input, no map selected.")
            return None, False

# Main execution logic
maps = list_all_maps()

if len(maps) > 0:
    # If maps exist, list them
    for map_obj in maps:
        arcpy.AddMessage(f"Existing map: {map_obj.name}")
    result_map, action_taken = rename_or_create_map()

    if action_taken:
        summary_message = f"Map '{result_map.name}' has been renamed to {county}_County."
    else:
        summary_message = "No action taken."
else:
    # No maps exist, inform the user
    result_map, action_taken = rename_or_create_map()
    if action_taken:
        summary_message = f"Map has been renamed to {county}_County."
    else:
        summary_message = "No map created."

# Print summary message
print(summary_message)
arcpy.AddMessage(summary_message)

#### Add GeoTIFFs

<font color="blue">**Add GeoTIFF files to the active map:**</font>

*How It Works:*
- *The script scans the county directory for valid GeoTIFF files, excluding those with "TMorth" in the filename.*
- *It matches the filenames with a list of quadrangle names and adds the corresponding GeoTIFF files to the active map.*

*The order of the quads in the list determines the order in which the GeoTIFFs are added to the map. The last item in the list is added first, with the topmost GeoTIFF covering the northwest corner of the county and the bottommost covering the southeast corner.*

In [ ]:
import arcpy
import os
import time  # Import time module to add a delay

def add_geotiffs_to_map(county_dir, quads, map_obj, delay_seconds=1):
    # Ensure the county directory exists
    if not os.path.isdir(county_dir):
        print(f"The directory {county_dir} does not exist.")
        return

    # Step 1: Get all GeoTIFF files within the county directory
    valid_geotiffs = []

    # Walk through the county directory to find all .tif files excluding those with "TMorth" in the filename
    for dirpath, dirnames, filenames in os.walk(county_dir):
        for filename in filenames:
            if filename.lower().endswith('.tif') and "tmorth" not in filename.lower():
                valid_geotiffs.append(os.path.join(dirpath, filename))

    # Step 2: Provide feedback on the number of GeoTIFFs found
    if len(valid_geotiffs) == 0:
        print("No valid GeoTIFFs found in the directory.")
    else:
        print(f"Found {len(valid_geotiffs)} valid GeoTIFF(s). Adding to the map...")

        added_geotiffs = set()
        for quad in quads:
            # Normalize the quad name by removing spaces and converting to lowercase
            quad_normalized = quad.replace(" ", "").lower()

            # Find matching GeoTIFFs
            matching_geotiffs = [geotiff for geotiff in valid_geotiffs 
                                 if quad_normalized in os.path.basename(geotiff).replace(" ", "").lower()]

            # Add the matching GeoTIFF to the map (if it's not already added)
            for geotiff in matching_geotiffs:
                if geotiff not in added_geotiffs:
                    try:
                        # Normalize path to always use forward slashes
                        geotiff = geotiff.replace("\\", "/")  # Convert backslashes to forward slashes
                        
                        # Handle spaces in GeoTIFF filenames by ensuring they're properly processed
                        if " " in os.path.basename(geotiff):
                            print(f"GeoTIFF with spaces detected: {geotiff}")
                        
                        map_obj.addDataFromPath(geotiff)
                        added_geotiffs.add(geotiff)  # Track as added
                        print(f"Added {geotiff} to the map.")
                        
                        # Introduce a time lag before adding the next GeoTIFF
                        time.sleep(delay_seconds)  # Delay in seconds

                    except Exception as e:
                        print(f"Error adding {geotiff} to the map: {e}")

        print(f"{len(added_geotiffs)} GeoTIFFs added to the map.")

# Assuming the map object is already available in ArcGIS Pro
# Get the current map from ArcGIS Pro
aprx = arcpy.mp.ArcGISProject("CURRENT")
map_obj = aprx.activeMap  # Get the active map in the project

# Check if the directory exists and add GeoTIFFs to the map
if os.path.isdir(county_directory):
    add_geotiffs_to_map(county_directory, quads, map_obj, delay_seconds=1)  # Set delay to 2 seconds between each addition
else:
    print(f"The county directory {county_directory} does not exist.")

<font color="blue">**Group GeoTIFFs in the Contents pane:**</font>

*How It Works:*
- *The script checks if a "topos" group already exists in the active map; if not, it creates one.*
- *It loops through all layers in the map, adding all .tif raster layers to the "topos" group in reverse order.*
- *It removes any .tif layers that are not part of the "topos" group, cleaning up the map.*

In [ ]:
import arcpy

# Get the active map from the currently open project
aprx = arcpy.mp.ArcGISProject("CURRENT")  # Refers to the active project
map_obj = aprx.activeMap  # Get the active map

# Check if the map is valid
if map_obj is None:
    print("No active map found.")
else:
    # Create a new group layer named "topos" if it doesn't already exist
    group_name = "topos"
    existing_groups = [layer for layer in map_obj.listLayers() if layer.isGroupLayer]

    # Check if the "topos" group already exists, if not, create it
    topos_group = None
    for group in existing_groups:
        if group.name == group_name:
            topos_group = group
            break

    if topos_group is None:
        # Create a new group layer called "topos"
        topos_group = map_obj.createGroupLayer(group_name)
    
    # Loop through layers and add all GeoTIFFs (.tif) to the "topos" group in reverse order
    for layer in map_obj.listLayers():  # Reverse the list of layers
        if layer.isRasterLayer and layer.name.lower().endswith(".tif"):  # Check for .tif layers
            # Move the GeoTIFF layer to the "topos" group
            map_obj.addLayerToGroup(topos_group, layer)
    
    # Remove all ungrouped .tif layers (those not in the "topos" group)
    for layer in map_obj.listLayers():
        if layer.isRasterLayer and layer.name.lower().endswith(".tif"):
            # Check if the layer is not part of the "topos" group
            if layer not in topos_group.listLayers():
                # Remove the ungrouped .tif layer from the map
                map_obj.removeLayer(layer)
    
    print(f"All .tif layers have been grouped under '{group_name}' group in reversed order, and ungrouped layers have been removed.")

#### Add County Boundaries

<font color="blue">**Add the *ky_counties* feature class and update symbology to create a boundary outline of the county of interest:**</font>

*How It Works:*
- *The script checks if the specified feature class exists in the given geodatabase.*
- *It adds the feature class to the active map and customizes its symbology with a transparent fill, colored outline, and offset effect.*
- *It applies a definition query to filter the layer based on the given county name and saves the project.*

In [ ]:
import arcpy
import os

# Specify the feature class name you are looking for
fc_name = "ky_counties"

# Search for the feature class in the geodatabase
if gdb_path and os.path.isdir(gdb_path):
    # Construct the path to the feature class
    fc_path = os.path.join(gdb_path, fc_name)
    
    if arcpy.Exists(fc_path):
        print(f"Feature class '{fc_name}' found in {gdb_path}.")
        
        # Get the active project and map
        aprx = arcpy.mp.ArcGISProject("CURRENT")
        map_obj = aprx.activeMap

        # Add the feature class to the map
        new_layer = map_obj.addDataFromPath(fc_path)
        
        # Apply the symbology with a SimpleRenderer
        sym = new_layer.symbology
        sym.updateRenderer('SimpleRenderer')  # Use the SimpleRenderer
        
        # Apply the symbol settings
        symbol = sym.renderer.symbol
        
        # Set color to transparent fill
        symbol.color = {'RGB': [0, 0, 0, 0]}  # Transparent fill (alpha 0)
        
        # Set outline color (red = 169, green = 0, blue = 230)
        symbol.outlineColor = {'RGB': [169, 0, 230, 100]}  # Outline: RGB(169, 0, 230) with full opacity
        
        # Set outline width to 5 points
        symbol.outlineWidth = 5  # Outline width set to 5 points
        
        # Apply an outline offset effect (5 pts)
        symbol.outlineOffsetX = 5  # Horizontal offset
        symbol.outlineOffsetY = 5  # Vertical offset
        
        # Apply the updated symbology back to the layer
        new_layer.symbology = sym
        
        # Apply the definition query based on the "county" variable
        new_layer.definitionQuery = f"NAME = '{county}'"
        
        print(f"Definition query applied: NAME = '{county}'")
        
        # Save the project after changes
        aprx.save()
    else:
        print(f"Feature class '{fc_name}' does not exist in the specified geodatabase.")
else:
    print("Invalid geodatabase path. Please check and try again.")

#### Create Cemetery Polygon Feature Class

<font color="blue">**Create a copy of *Cemetery_PolygonFC_Template*, rename, and change symbology:**</font>

*How It Works:*
- *The script copies the "Cemetery_PolygonFC_Template" feature class to a new feature class, named after the county.*
- *It checks if the new layer is already in the map; if not, it adds the new feature class to the map.*
- *It applies custom symbology to the new layer, setting it to a semi-transparent bright pink with a transparent outline, then saves the project.*

In [ ]:
import arcpy
import os

# Define the source feature class and the new name
source_fc = "Cemetery_PolygonFC_Template"
new_fc_name = f"{county}_County"

# Construct the full paths for source and destination feature classes
source_fc_path = os.path.join(gdb_path, source_fc)
new_fc_path = os.path.join(gdb_path, new_fc_name)

# Check if the source feature class exists
if arcpy.Exists(source_fc_path):
    # Copy the feature class to the new location with the new name
    arcpy.CopyFeatures_management(source_fc_path, new_fc_path)
    print(f"Polygon cemetery feature class created for {county} County: '{new_fc_name}'.")
else:
    print(f"Feature class '{source_fc}' does not exist in the specified geodatabase.")

# Get the map and check if the layer is already present before adding it
aprx = arcpy.mp.ArcGISProject("CURRENT")
map_obj = aprx.activeMap

# Check if the layer is already in the map, and if not, add it
layer_exists = False
for layer in map_obj.listLayers():
    if layer.name == new_fc_name:
        layer_exists = True
        break

if not layer_exists:
    new_layer = map_obj.addDataFromPath(new_fc_path)
    print(f"Added '{new_fc_name}' to the map.")
else:
    print(f"Layer '{new_fc_name}' is already in the map.")

# Access the symbology of the new layer
new_layer = map_obj.listLayers(new_fc_name)[0]
sym = new_layer.symbology

# Set the symbology to a SimpleRenderer
sym.updateRenderer("SimpleRenderer")

# Apply the semi-transparent bright pink color (Red 255, Green 0, Blue 197, 45% transparency)
sym.renderer.symbol.color = {'RGB': [255, 0, 197, 150]}  # 45% transparency (alpha = 115)

# Make the outline completely transparent
sym.renderer.symbol.outlineColor = {'RGB': [255, 255, 255, 0]}  # Transparent outline (alpha = 0)

# Apply the updated symbology to the layer
new_layer.symbology = sym

# Save the project after making changes
aprx.save()

print(f"Symbology applied: Semi-transparent bright pink fill with a transparent outline.")

#### Add Cemetery Point Data

<font color="blue">**Add *KYCemPts_FindaGrave_01292025*, *Archaeology_HistoricCem_Centerpoints*, and *KYCemPts_FindaGrave_01292025* feature classes to the map with custom symbology:**</font>

*How It Works:*
- *The script checks if each feature class exists in the geodatabase and adds it to the active map.*
- *It applies specific symbology based on the feature class name, using symbols such as 3D pushpins and circle markers.*
- *After applying symbology to the layers, the script saves the ArcGIS project.*

In [ ]:
import arcpy
import os

# Define the feature classes
feature_classes = [
    "KY_CemData_Draft2013",
    "Archaeology_HistoricCem_Centerpoints",
    "KYCemPts_FindaGrave_01292025"
]

# Get the active ArcGIS project and map
aprx = arcpy.mp.ArcGISProject("CURRENT")  # Get the current ArcGIS Project
map_obj = aprx.activeMap  # Get the active map

# Function to apply symbology to point feature classes (for color and outline)
def apply_point_symbology(layer, fill_color, outline_color, outline_width, size):
    sym = layer.symbology
    sym.updateRenderer("SimpleRenderer")  # Apply the SimpleRenderer for point features
    
    # Set the fill color (using RGB)
    sym.renderer.symbol.color = {'RGB': fill_color}
    
    # Set the outline color (using RGB)
    sym.renderer.symbol.outlineColor = {'RGB': outline_color}
    
    # Set the outline width
    sym.renderer.symbol.outlineWidth = outline_width
    
    # Set the point size
    sym.renderer.symbol.size = size
    
    # Apply the changes to the layer's symbology
    layer.symbology = sym

# Function to apply symbol from gallery (for circle symbol)
def apply_circle_symbol(layer):
    sym = layer.symbology
    sym.updateRenderer("SimpleRenderer")  # Apply the SimpleRenderer for point features
    
    # Apply a circle symbol from the symbol gallery
    sym.renderer.symbol.applySymbolFromGallery("Circle 6")
    
    # Apply the changes to the layer's symbology
    layer.symbology = sym

# Function to apply 3D Billboard pushpin symbols
def apply_pushpin_symbol(layer, symbol_name):
    sym = layer.symbology
    sym.updateRenderer("SimpleRenderer")  # Apply the SimpleRenderer for point features
    
    # Apply the pushpin symbol from the gallery (Style: 3D Billboards)
    sym.renderer.symbol.applySymbolFromGallery(symbol_name)
    
    # Apply the changes to the layer's symbology
    layer.symbology = sym

# Loop through each feature class and add them to the map with appropriate symbology
for fc in feature_classes:
    fc_path = os.path.join(gdb_path, fc)  # Get the full path to the feature class
    
    if arcpy.Exists(fc_path):
        # Add the feature class to the map
        layer = map_obj.addDataFromPath(fc_path)
        
        # Apply specific symbology based on the feature class name
        if fc == "KY_CemData_Draft2013":
            # Apply "Green Pushpin 1" from 3D Billboards style
            apply_pushpin_symbol(layer, "Green Pushpin 1")
        
        elif fc == "Archaeology_HistoricCem_Centerpoints":
            # Apply "Blue Pushpin 1" from 3D Billboards style
            apply_pushpin_symbol(layer, "Blue Pushpin 1")
        
        elif fc == "KYCemPts_FindaGrave_01292025":
            # Apply Circle 6 symbol from the gallery
            apply_circle_symbol(layer)
        
        print(f"Added and symbolized {fc} to the map.")
    else:
        print(f"Feature class '{fc}' does not exist in the geodatabase.")

# Optionally, save the project after making changes
aprx.save()

## 2️⃣Digitization

Manually digitize cemetery polygons using the topographic maps. (The point feature class will be created later.) Review topographic map symbols with this [map key](https://www.historicaerials.com/topo-map-key) or consult the [USGS documentation](https://pubs.usgs.gov/gip/TopographicMapSymbols/topomapsymbols.pdf).

Features in the *Archaeology_HistoricCem_Centerpoints* and *KY_CemData_Draft2013* feature classes should also be digitized.

* If a feature is near or overlaps a cemetery digitized from a topographic map, include a note in the "Comments" field. For example:
    * _Also included in archaeological data from USFS Daniel Boone National Forest._
    * _Also included in 2013 cemetery data from USFS Daniel Boone National Forest._
* If a feature is not near or overlapping a cemetery, approximate the cemetery size and create a polygon with the point in the center. Add a note in the "Comments" field. For example:
    * _Digitized using archaeological cemetery center point data from USFS Daniel Boone National Forest._
    * _Digitized using 2013 cemetery point data from USFS Daniel Boone National Forest._

<span style="color:blue">**(1) After digitizing all cemeteries for a quadrangle, populate the *Topo_Name* field.**</span>

<span style="color:blue">**(2) Before proceeding to attribute data entry, save save your edits and save the project to avoid losing any work.**</span>

## 3️⃣Attribute Data Entry

### Automated

<font color="blue">**Populate the *County_Name* field:**</font>

In [ ]:
arcpy.management.CalculateField(
    in_table=f"{county}_County",
    field="County_Name",
    expression=f'"{county}"',
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<font color="blue">**Populate the *Latitude* and *Longitude* fields:**</font>

In [ ]:
arcpy.management.CalculateGeometryAttributes(
    in_features=f"{county}_County",
    geometry_property="Latitude CENTROID_Y;Longitude CENTROID_X",
    length_unit="",
    area_unit="",
    coordinate_system=None,
    coordinate_format="SAME_AS_INPUT"
)

<font color="blue">**Populate the *Zip* field using [U.S. Postal Service ZIP Code Boundaries](https://www.arcgis.com/home/item.html?id=5f31109b46d541da86119bd4cf213848):**</font>

*How It Works:*
- *The script checks if the *zip_zones* feature class exists in the geodatabase and performs a spatial join.*
- *It searches for a field with "AddSpatialJoin" in its name and "ZIP Code" in its alias to dynamically identify the relevant field for calculation.*
- *After calculating the field values, the script removes the spatial join and prints status messages throughout the process.*

In [ ]:
import arcpy
import os

# Set the workspace to the correct geodatabase path
arcpy.env.workspace = f"{gdb_path}"  # Adjust the path accordingly

# Check if "zip_zones" exists in the workspace
zip_zones_path = os.path.join(arcpy.env.workspace, "zip_zones")
print(f"Checking if {zip_zones_path} exists...")
if arcpy.Exists(zip_zones_path):
    print(f"{zip_zones_path} exists.")
else:
    print(f"ERROR: {zip_zones_path} does not exist.")
    raise Exception(f"ERROR: {zip_zones_path} does not exist.")

# Proceed if 'zip_zones' exists
print(f"Starting spatial join for {county}_County with zip_zones...")

arcpy.management.AddSpatialJoin(
    target_features=f"{county}_County",
    join_features=zip_zones_path,  # Ensure it's the full path
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    field_mapping='ZIP_CODE "ZIP Code" true true false 10 Text 0 0,First,#,zip_zones,ZIP_CODE,0,9;PO_NAME "Post Office Name" true true false 100 Text 0 0,First,#,zip_zones,PO_NAME,0,99;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,zip_zones,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,zip_zones,Shape_Area,-1,-1',
    match_option="HAVE_THEIR_CENTER_IN",
    search_radius=None,
    distance_field_name="",
    permanent_join="NO_PERMANENT_FIELDS",
    match_fields=None
)

# List all fields in the table
fields = arcpy.ListFields(f"{county}_County")
print(f"Fields in {county}_County: {', '.join([field.name for field in fields])}")

# Initialize a variable to hold the field name
zip_code_field = None

# Loop through the fields and check for the conditions
print(f"Searching for the field with 'AddSpatialJoin' in the name and 'ZIP Code' in the alias...")
for field in fields:
    # Check if the field name contains "AddSpatialJoin" and the alias contains "ZIP Code"
    if "AddSpatialJoin" in field.name and "ZIP Code" in field.aliasName:
        zip_code_field = field.name
        print(f"Found matching field: {zip_code_field}")
        break  # Exit the loop once we find the matching field

# Check if we found the correct field
if zip_code_field is None:
    print("ERROR: Field with 'AddSpatialJoin' in the name and 'ZIP Code' in the alias not found.")
    raise Exception("Field with 'AddSpatialJoin' in the name and 'ZIP Code' in the alias not found.")

# Print out the field name for debugging purposes
print(f"ZIP code field identified: {zip_code_field}")

# Now use the dynamically found field name in the CalculateField expression
print(f"Calculating field values for {county}_County.Zip using field {zip_code_field}...")
arcpy.management.CalculateField(
    in_table=f"{county}_County",
    field=f"{county}_County.Zip",  # Assuming you want to populate the 'Zip' field in your table
    expression=f"!{zip_code_field}!",  # Use the dynamically found field name
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

# Remove spatial join
join_name = zip_code_field.split('.')[0]  # Assuming split at period is a safe approach for the join name
print(f"Removing spatial join: {join_name} from {county}_County")

arcpy.management.RemoveJoin(
    in_layer_or_view=f"{county}_County",
    join_name=f"{join_name}"
)

# Optional: Inform user that the process is completed
print("Spatial join and field calculation completed successfully. Spatial join removed.")

### Manual

**Populate remaining attribute fields** (*Cemetery_Name1*, *Cemetery_Name2*, *Street_Address*, *City*, *Comments*, *Burial_Number*, *Link*). Note that some attribute fields may not have valid values available. If you are unable to find a valid value, leave the field blank and/or make a note in the "Comments" field.

Sources:
- [Google Maps](https://www.google.com/maps)
- [OpenStreetMap](https://www.openstreetmap.org/)
- [Kentucky Natural Color Imagery (2024)](https://kygeonet.maps.arcgis.com/home/webmap/viewer.html)
- *KYCemPts_FindaGrave_01292025* feature class
- Find a Grave [website](https://www.findagrave.com/cemetery)
- [Regrid parcel data](https://app.regrid.com/us/ky)
- [Cemeteries in Kentucky Database](https://kyhistory.com/digital/collection/LIB/id/493/)

*See additional sources in the Data Sources section.*

Use your own judgment in assigning point attributes to digitized polygon features. **The Find a Grave points may or may not overlap with digitized cemeteries; in some cases, a point may not overlap but will be in close proximity to the polygon.**

Even if no Find a Grave point is near a cemetery of interest, Google Maps, OpenStreetMap, or Regrid parcel data may show a cemetery name. If this occurs, you may be able to find a Find a Grave match by searching the website and matching based on a unique cemetery name, address, or description. (Even potential matches can be linked in the "Comments" field.)

<font color="blue">**Input your search term and automatically open the Find a Grave search URL in your default browser:**</font>

In [ ]:
import webbrowser

# Define the red color using ANSI escape codes
RED = '\033[91m'
RESET = '\033[0m'  # Reset color back to default

# Print the instructions in red, split across two lines
print(f"{RED}Note: Upper or lowercase does not matter.\nYou can enter any part of the cemetery name (partial or full).\nFor example, to search for 'Pleasant Valley Memorial Gardens', you could enter 'pleasant', 'valley', or 'gardens'.{RESET}")

# Prompt the user for the cemetery name
cemetery_name = input("Please enter the cemetery name: ")

# Format the URL with the user's input and the county variable
url = f"https://www.findagrave.com/cemetery/search?cemetery-name={cemetery_name}&cemetery-loc={county}+County%2C+Kentucky%2C+USA&only-with-cemeteries=cemOnly&locationId=county_1086"

# Open the URL in the default web browser
webbrowser.open(url)

<font color="blue">**If you want to check unmatched/unknown cemeteries for headstones, nearby landmarks, or addresses in Google Maps:**</font> <span style="color:red">*Run the next cell AFTER copying the approximate centroid coordinates of a digitized polygon. (Right-click the center of the cemetery in the map, select "Convert Coordinates", then select "DD".)*</span>

In [ ]:
import webbrowser, tkinter

def visual_check():
    win = tkinter.Tk()
    win.withdraw()
    coordinates = win.clipboard_get()
    webbrowser.open(f"https://www.google.com/maps/place/{coordinates}")    

visual_check()

## 4️⃣Metadata Creation

After digitizing all cemeteries and completing the attribute data entry, the next step is to add metadata to the feature classes. Proper metadata ensures the data is well-documented, easily understood, and can be efficiently shared and maintained. Follow the instructions below to complete the metadata entry process.

<font color="blue">**Run the cell below to ensure your name appears in the credits.**</font>

In [ ]:
name = input("Please enter your full name (as it will appear in the credits): ")

<font color="blue">**Create a cemetery point feature class:**</font>

In [ ]:
arcpy.management.FeatureToPoint(
    in_features=f"{county}_County",
    out_feature_class=f"{gdb_path}\{county}_County_Points",
    point_location="CENTROID"
)

If you have encounter an issue running the code below, check if the metadata is read-only:
- Right-click the feature class in the **Contents** pane.
- Select **Properties**.
- Go to the **Metadata** tab.
- Click the dropdown and select "Layer has its own metadata".
- Save by selecting **Apply**/**OK**.
- You may also need to restart ArcGIS Pro.

<font color="blue">**Edit the metadata of the point feature class:**</font>

*How It Works:*
- *The script uses a regex pattern to extract the state abbreviation, place name, and year from GeoTIFF filenames and adds these details to a list.*
- *It searches for a feature class layer ({county}_County_Points) in the active map and checks if the metadata can be updated.*
- *It updates the metadata for the feature class with new title, summary, description, tags, and credits, using the extracted GeoTIFF details and predefined metadata fields.*

In [ ]:
import os
import re
import arcpy
from arcpy import metadata as md

# Define the topo_list
topo_list = []

# Regex pattern to match the filenames
pattern = r'([A-Za-z]{2})_([^_]+)_\d{6}_(\d{4})_\d+_geo\.tif'

# Loop through all files in the specified directory
for filename in os.listdir(county_directory):
    # Check if the file is a GeoTIFF (ends with .tif)
    if filename.lower().endswith('.tif'):
        # Use regex to match the filename and extract details
        match = re.match(pattern, filename)
        if match:
            state_abbr = match.group(1)  # State abbreviation (KY, IN, etc.)
            place_name = match.group(2)  # Place name (Claysville, Mount Olivet, etc.)
            year = match.group(3)  # Year (e.g., 1952)
            
            # Construct the topo list entry
            topo_list.append(f"{state_abbr} {place_name} {year}")
        else:
            print(f"Filename does not match the expected pattern: {filename}")

# If no topographic files are found
if not topo_list:
    print("No topographic maps found in the specified directory.")
else:
    print(f"Found {len(topo_list)} topographic map(s).")

# Convert topo_list to a comma-separated string
topo_list_str = ', '.join(topo_list)

# Get the active ArcGIS project
aprx = arcpy.mp.ArcGISProject("CURRENT")
current_map = aprx.activeMap

# Find the feature class layer with name matching "{county}_County_Points"
feature_layer = None
for layer in current_map.listLayers():
    if layer.isFeatureLayer and layer.name == f"{county}_County_Points":
        feature_layer = layer
        break

# Check if a matching feature layer was found
if feature_layer:
    # Get the Metadata object for the feature class
    feature_md = md.Metadata(feature_layer)
    
    # Construct the new title
    new_title = f"{county} County Historical Cemeteries"
    
    # Set the new tags
    new_tags = f"cemetery, Kentucky, {county} County, historical, grave"

    # Set the new summary
    new_summary = (f"This point feature class serves as a record of historical cemeteries "
                   f"within {county} County, Kentucky, as digitized from historical "
                   f"USGS georeferenced topographic maps, Find A Grave, 2013 cemetery point data from USFS "
                   f"Daniel Boone National Forest, and archaeological cemetery center point data from USFS "
                   f"Daniel Boone National Forest.")

    # Set the new description
    new_description = (f"Cemetery polygons are digitized by overlying historic USGS georeferenced topographic maps "
                       f"and drawing polygons around cemetery locations on the historic topographic maps. Point data "
                       f"provided by USFS Daniel Boone National Forest is also used as reference points for cemetery "
                       f"locations, with polygons digitized over the point locations with the points at the center. "
                       f"The schema for each feature class is filled in with available information from county "
                       f"historical societies, the Find A Grave website, and other internet research.\n\n"
                       f"USGS georeferenced topographic maps used (1:24,000 scale):\n\n{topo_list_str}.")

    # Set the fixed credits
    fixed_credits = f"United States Forest Service Daniel Boone National Forest, Kentucky Heritage Council, State of Kentucky Commonwealth Office of Technology GIS, {name}"

    # Update the metadata
    if not feature_md.isReadOnly:
        feature_md.title = new_title
        feature_md.summary = new_summary  # Set the summary to the new format
        feature_md.description = new_description  # Set the description
        feature_md.tags = new_tags  # Set tags to the predefined list
        feature_md.credits = fixed_credits  # Fixed credits
        
        # Save changes
        feature_md.save()
        print("Metadata updated successfully!")
    else:
        print("Metadata is read-only and cannot be updated.")
else:
    print(f"No feature class found with the name '{county}_County_Points'.")

<font color="blue">**Edit the metadata of the polygon feature class:**</font>

In [ ]:
import os
import re
import arcpy
from arcpy import metadata as md

# Define the topo_list
topo_list = []

# Regex pattern to match the filenames
pattern = r'([A-Za-z]{2})_([^_]+)_\d{6}_(\d{4})_\d+_geo\.tif'

# Loop through all files in the specified directory
for filename in os.listdir(county_directory):
    # Check if the file is a GeoTIFF (ends with .tif)
    if filename.lower().endswith('.tif'):
        # Use regex to match the filename and extract details
        match = re.match(pattern, filename)
        if match:
            state_abbr = match.group(1)  # State abbreviation (KY, IN, etc.)
            place_name = match.group(2)  # Place name (Claysville, Mount Olivet, etc.)
            year = match.group(3)  # Year (e.g., 1952)
            
            # Construct the topo list entry
            topo_list.append(f"{state_abbr} {place_name} {year}")
        else:
            print(f"Filename does not match the expected pattern: {filename}")

# If no topographic files are found
if not topo_list:
    print("No topographic maps found in the specified directory.")
else:
    print(f"Found {len(topo_list)} topographic map(s).")

# Convert topo_list to a comma-separated string
topo_list_str = ', '.join(topo_list)

# Get the active ArcGIS project
aprx = arcpy.mp.ArcGISProject("CURRENT")
current_map = aprx.activeMap

# Find the feature class layer with name matching "{county}_County"
feature_layer = None
for layer in current_map.listLayers():
    if layer.isFeatureLayer and layer.name == f"{county}_County":
        feature_layer = layer
        break

# Check if a matching feature layer was found
if feature_layer:
    # Get the Metadata object for the feature class
    feature_md = md.Metadata(feature_layer)
    
    # Construct the new title
    new_title = f"{county} County Historical Cemeteries"
    
    # Set the new tags
    new_tags = f"cemetery, Kentucky, {county} County, historical, grave"

    # Set the new summary
    new_summary = (f"This polygon feature class serves as a record of historical cemeteries "
                   f"within {county} County, Kentucky, as digitized from historical "
                   f"USGS georeferenced topographic maps, Find A Grave, 2013 cemetery point data from USFS "
                   f"Daniel Boone National Forest, and archaeological cemetery center point data from USFS "
                   f"Daniel Boone National Forest.")

    # Set the new description
    new_description = (f"Cemetery polygons are digitized by overlying historic USGS georeferenced topographic maps "
                       f"and drawing polygons around cemetery locations on the historic topographic maps. Point data "
                       f"provided by USFS Daniel Boone National Forest is also used as reference points for cemetery "
                       f"locations, with polygons digitized over the point locations with the points at the center. "
                       f"The schema for each feature class is filled in with available information from county "
                       f"historical societies, the Find A Grave website, and other internet research.\n\n"
                       f"USGS georeferenced topographic maps used (1:24,000 scale):\n\n{topo_list_str}.")

    # Set the fixed credits
    fixed_credits = f"United States Forest Service Daniel Boone National Forest, Kentucky Heritage Council, State of Kentucky Commonwealth Office of Technology GIS, {name}"

    # Update the metadata
    if not feature_md.isReadOnly:
        feature_md.title = new_title
        feature_md.summary = new_summary  # Set the summary to the new format
        feature_md.description = new_description  # Set the description
        feature_md.tags = new_tags  # Set tags to the predefined list
        feature_md.credits = fixed_credits  # Fixed credits
        
        # Save changes
        feature_md.save()
        print("Metadata updated successfully!")
    else:
        print("Metadata is read-only and cannot be updated.")
else:
    print(f"No feature class found with the name '{county}_County'.")

## 5️⃣Sharing

<font color="blue">**Create a new geodatabase and copy point and polygon feature classes from the default geodatabase into it:**</font>

*How It Works:*
- *The script uses Tkinter to prompt the user to select a folder for the new geodatabase.*
- *It checks if the new geodatabase already exists. If not, the script creates the geodatabase and copies the relevant feature classes from the default geodatabase into it.*

*This code may add the feature class copies to the current map. The copies (i.e., the layers that should be removed) will be listed before (or on top of) the originals in the Contents pane.*

In [ ]:
import arcpy
import os
import tkinter as tk
from tkinter import filedialog

# Initialize Tkinter root for file dialog
root = tk.Tk()
root.withdraw()  # Hide the root window

# Ask the user to choose a folder where to save the new geodatabase
folder_path = filedialog.askdirectory(title=f"Select a Folder to Save {county}_County.gdb")

# Check if the user selected a folder
if folder_path:
    # Define the new geodatabase name
    gdb_name = f"{county}_County.gdb"
    
    # Define the full path to the new geodatabase
    new_gdb_path = os.path.join(folder_path, gdb_name)

    # Normalize the path to use consistent slashes
    new_gdb_path = new_gdb_path.replace("\\", "/")  # Replace backslashes with forward slashes

    # Check if the geodatabase already exists, if not create it
    if not arcpy.Exists(new_gdb_path):
        arcpy.CreateFileGDB_management(folder_path, gdb_name)
        print(f"New geodatabase created: {new_gdb_path}")
    else:
        print(f"Geodatabase {gdb_name} already exists at the specified location.")

    # Get the default geodatabase from the environment
    default_gdb = arcpy.env.workspace  # This is the default geodatabase set in your ArcGIS environment

    # List all feature classes in the default geodatabase
    feature_classes = arcpy.ListFeatureClasses()

    # Check if there are feature classes to process
    if not feature_classes:
        print("No feature classes found in the default geodatabase.")
    else:
        # Loop through the feature classes
        for fc in feature_classes:
            # Check if the feature class name contains the county variable
            if county.lower() in fc.lower():  # Case-insensitive check
                try:
                    # Define the destination path for the feature class
                    destination_fc = os.path.join(new_gdb_path, fc)

                    # Normalize the path to use consistent slashes
                    destination_fc = destination_fc.replace("\\", "/")  # Replace backslashes with forward slashes

                    # Check if the feature class already exists in the new geodatabase
                    if arcpy.Exists(destination_fc):
                        print(f"Feature class {fc} already exists in the new geodatabase. Skipping...")
                    else:
                        # Copy the feature class to the new geodatabase
                        arcpy.CopyFeatures_management(fc, destination_fc)
                        print(f"Feature class {fc} copied successfully to {new_gdb_path}.")
                except Exception as e:
                    print(f"Error copying feature class {fc}: {e}")
else:
    print("No folder was selected. The process has been cancelled.")

<font color="blue">**Save the project, close ArcGIS Pro, then:**</font>

1. Navigate to the new geodatabase location using File Explorer (refer to the path in the output message above).
2. Compress the geodatabase into a ZIP file.
3. Email the ZIP file to the person currently organizing the project data.